In [ ]:
import torch.nn as nn
import torch

In [ ]:
class Q_critic(nn.Module):
    def __init__(self):
        super(Q_critic, self).__init__()
        self.line1=nn.Linear(10,256)
        self.line2=nn.Linear(256,256)
        self.line3=nn.Linear(256,1)
        
        
        self.relu=nn.ReLU()
        
    def forward(self, y,act):
        x=torch.cat((y,act),dim=-1)
        x=self.line1(x)
        x=self.relu(x)
        x=self.line2(x)
        x=self.relu(x)
        x=self.line3(x)
        
        return x

In [ ]:
class actor(nn.Module):
    def __init__(self):
        super(actor, self).__init__()
        self.line1=nn.Linear(8,256)
        self.line2=nn.Linear(256,256)
        
        
        self.relu=nn.ReLU()
        
        self.mean = nn.Linear(256, 2)
        self.tanh=nn.Hardtanh()
        
        self.std = nn.Linear(256, 2)
        self.softplus=nn.Softplus()
        
    def forward(self, x):
        x=self.line1(x)
        x=self.relu(x)
        x=self.line2(x)
        x=self.relu(x)
        
        mean=self.mean(x)
        mean=self.tanh(mean)  # tanh activation to bound the actions

        std = self.std(x)
        std = self.softplus(std) # Convert log std to std

        dist = torch.distributions.Normal(mean, std)
        
        action = dist.sample()
        adjusted_action = torch.tanh(action)
        
        log_prob = dist.log_prob(action)# 샘플링된 행동의 로그 확률 계산
        adjusted_log_prob=log_prob-torch.log(1-adjusted_action.pow(2) + 1e-6)
        
        return adjusted_action, adjusted_log_prob